# Notebook de départ

Ce notebook est un point de départ qui vous montre la base de comment accéder aux données et en extraire la distribution de la pseudo-rapidité $\eta$ des traces.

## Importation de modules python

La plupart des notebooks jupyter commencent par une liste d'importation de modules python qui sont ensuite utilisés.
Dans ce projet les modules de base sont : 
- [uproot](https://uproot.readthedocs.io/en/latest/) pour lire (et écrire) des données au format [ROOT](https://root.cern.ch) (mais sans avoir besoin d'installer ROOT)
- [awkward](https://awkward-array.readthedocs.io/en/latest/) pour manipuler des tableaux avec des structures de taille variable (qui sont ce que l'on utilise couramment en physique des hautes énergies, e.g. chaque événement contient potentiellement un nombre différent de muons)
- [numpy](https://numpy.org) qui est la base de la base des modules de "scientific computing" dans le monde Python. Un peu moins nécessaire dans notre cas (les awkward arrays sont généralement plus adaptés) mais certainement un module à connaître et à explorer dans votre future carrière.
- [matplotlib](https://matplotlib.org) pour créer de nombreux types de visualisations
- [hist](https://hist.readthedocs.io/en/latest/index.html) pour créer des histogrammes
- [vector](https://vector.readthedocs.io/en/latest/index.html) pour manipuler des vecteurs 2D, 3D et Lorentz.

Pour apprendre et pratiquer ces modules, voir par exemple un [tutorial récent](https://github.com/jpivarski-talks/2021-09-13-star-uproot-awkward-tutorial/blob/main/tutorial.ipynb)


In [ ]:
import uproot
import awkward as ak
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import hist
import vector
print("uproot version",uproot.__version__)
print("awkward version",ak.__version__)
print("numpy version",np.__version__)
print("matplotlib version",matplotlib.__version__)
print("hist version",hist.__version__)
print("vector version",vector.__version__)

## Emplacement des données

Les données à analyser/explorer/visualiser sont dans le répertoire `/pbs/throng/training/nantes-m2-rps-exp/data`. Pour chaque run, il y a deux fichiers, l'un correspondant aux vraies données,  `runXXX.data.root`, l'autre à des données simulées (dans les mêmes conditions que les données réelles), `runXXX.mc.root`.

In [ ]:
!ls -alh /pbs/throng/training/nantes-m2-rps-exp/data/*291263*

## Exploration rapide des données (i.e. le format des données)

In [ ]:
file = uproot.open("/pbs/throng/training/nantes-m2-rps-exp/data/run291263.data.root")

In [ ]:
file.keys()

In [ ]:
events = file["eventsTree"]
events.show()

L'idée est ensuite de mettre les variables d'intérêt dans des tableaux que l'on manipulera ensuite. Par exemple on récupère ci-dessous des tableaux contenant le nombre de traces par événement, les composantes x,y,z de l'impulsion des traces et leur charge; et ceci pour les 10000 premiers événements (pour que le reste de ce notebook d'exemple ne soit pas trop lent). 

In [ ]:
m = events.arrays(["nMuons","Muon_Px","Muon_Py","Muon_Pz","Muon_Charge"],how="zip",entry_stop=10000)
type(m)

Ce tableau peut être indexé de différentes façons. Avec un index unique pour accéder un seul élément : 

In [ ]:
m[401].to_list()

> Challenge : l'index 401 correspond en fait au premier événement qui a 5 muons dans cet échantillon de données. Comment trouver cet index rapidement ? Combien y a-t'il d'événements avec 5 muons dans cet échantillon ? 

Un index négatif est relatif à la fin du tableau. Le dernier événement est : 

In [ ]:
m[-1].to_list()

L'index peut également décrire un intervalle, e.g. pour afficher les 3 premiers événements :  

In [ ]:
m[0:3].to_list()

Un index peut également être un tableau de booléen où les valeurs à True indiquent les valeurs à indexer. Par exemple pour afficher le premier événement qui possède au moins 2 muons :

In [ ]:
m[m.nMuons>1][0].to_list()

## Calcul de la pseudo-rapidité 

Définissons maintenant quelques fonctions pour calculer la pseudo-rapidité $\eta$ d'un tri-vecteur... 

In [ ]:
def mag(px:float,py:float,pz:float):
    """Returns the norm of the 3-vector (px,py,pz)."""
    return math.sqrt(px*px+py*py+pz*pz)

def costheta(px:float,py:float,pz:float):
    """Returns the cos(theta) of the 3 vector (px,py,pz)."""
    ptot = mag(px,py,pz);
    return 1.0 if ptot == 0.0 else pz/ptot

def eta(px:float,py:float,pz:float):
    """Returns the pseudo-rapidity of the 3 vector (px,py,pz)."""
    ct = costheta(px,py,pz)
    if ct*ct < 1:
        return -0.5* math.log( (1.0-ct)/(1.0+ct) )
    if pz == 0: 
        return 0
    if pz>0: 
        return 10E10
    else:
        return -10E20


Puis on créé un tableau des valeurs de $\eta$ pour tous les muons (notez que l'on sélectionne les événements qui ont au moins un muon)

In [ ]:
etas = np.array([],dtype=float)
nofTracks=0 # il est toujours utile de compter ...
nofEvents=0 # 
for event in m[m.nMuons>0]:
    nofEvents+=1
    tracks = event["Muon"].to_list()
    for t in tracks:
        nofTracks+=1
        etas = np.append(etas,eta(t["Px"],t["Py"],t["Pz"])) 

In [ ]:
# ... et de vérifier que le compte est bon !
print(nofEvents,len(etas),nofTracks)

## Visualisation de la pseudo-rapidité des muons

In [ ]:
plt.hist(etas,bins=100);

La librarie [matplotlib](https://matplotlib.org) permet bien entendu de peaufiner cette figure autant que l'on souhaite... Par exemple en spécifiant un titre, en nommant les axes et en ajoutant une légende.

In [ ]:
fig = plt.figure(figsize=(5,3))
axes = fig.add_axes([0.1,0.1,0.9,0.9])
axes.set_title("$\eta$ distribution of muon tracks")
axes.set_xlabel("$\eta$")
axes.set_ylabel("# of tracks")
axes.set_yscale("log")
axes.hist(etas,bins=20,label="$\mu$",color='coral',histtype='step');
axes.legend();


## Exercices conseillés

Afin de vous familiariser avec les outils, essayez maintenant de faire un plot de l'impulsion transverse des muons. 

Puis, sur un même plot, tracer séparemment les $\mu^+$ et les $\mu^-$.

Enfin, observez la distribution du nombre de muons par événement et attaquez vous aux caractéristiques des paires de muons (masse invariante, impulsion transverse, rapidité, ...) !
Et, pour avoir une première idée de comment analyser efficacement un ensemble de données conséquent, jetez un oeil au [notebook suivant](./02-analyze-more-data.ipynb)

In [ ]:
plt.hist(m.nMuons,bins=[0,1,2,3,4,5,6,7,8,9,10])
plt.yscale("log")

In [ ]:
print("Nombre total d'événements: {}".format(len(m)))
n1 = len(m[m.nMuons>0])
n2 = len(m[m.nMuons>1])
n3 = len(m[m.nMuons>2])
n4 = len(m[m.nMuons>3])
n5 = len(m[m.nMuons>4])
n6 = len(m[m.nMuons>5])
n7 = len(m[m.nMuons>6])
n8 = len(m[m.nMuons>7])
print("Nombre d'événements avec au moins une trace: {}".format(n1))
print("Nombre d'événements avec au moins deux traces: {}".format(n2))
print("Nombre d'événements avec au moins trois traces: {}".format(n3))
print("Nombre d'événements avec au moins quatre traces: {}".format(n4))
print("Nombre d'événements avec au moins cinq traces: {}".format(n5))
print("Nombre d'événements avec au moins cinq traces: {}".format(n6))
print("Nombre d'événements avec au moins cinq traces: {}".format(n7))
print("Nombre d'événements avec au moins cinq traces: {}".format(n8))

In [ ]:
assert(n1+n2+n3+n4+n5+n6+n7+n8==nofTracks)